In [1]:
import import_ipynb
import pandas as pd
import numpy as np
from IPython.display import HTML, display

In [2]:
from P03_Imputation import matches
from P01_Pre_Processing import deliveries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1095 non-null   int64  
 1   season           1095 non-null   object 
 2   city             1044 non-null   object 
 3   date             1095 non-null   object 
 4   match_type       1095 non-null   object 
 5   player_of_match  1090 non-null   object 
 6   venue            1095 non-null   object 
 7   team1            1095 non-null   object 
 8   team2            1095 non-null   object 
 9   toss_winner      1095 non-null   object 
 10  toss_decision    1095 non-null   object 
 11  winner           1090 non-null   object 
 12  result           1095 non-null   object 
 13  result_margin    1076 non-null   float64
 14  target_runs      1092 non-null   float64
 15  target_overs     1092 non-null   float64
 16  super_over       1095 non-null   object 
 17  method        

,Match,Team 1,Team 2,Date,Time,Season
270,Match 19,Mumbai Indians,Delhi Capitals,2012-04-16,8:00 pm,2012
271,Match 20,Rajasthan Royals,Sunrisers Hyderabad,2012-04-17,4:00 pm,2012
272,Match 21,Royal Challengers Bangalore,Pune Warriors India,2012-04-17,8:00 pm,2012
273,Match 22,Punjab Kings,Kolkata Knight Riders,2012-04-18,8:00 pm,2012
274,Match 23,Delhi Capitals,Sunrisers Hyderabad,2012-04-19,4:00 pm,2012
275,Match 24,Chennai Super Kings,Pune Warriors India,2012-04-19,8:00 pm,2012
276,Match 25,Punjab Kings,Royal Challengers Bangalore,2012-04-20,8:00 pm,2012
277,Match 26,Chennai Super Kings,Rajasthan Royals,2012-04-21,4:00 pm,2012
278,Match 27,Delhi Capitals,Pune Warriors India,2012-04-21,8:00 pm,2012
279,Match 28,Mumbai Indians,Punjab Kings,2012-04-22,4:00 pm,2012


✅ Merged dataset shape: (1095, 24)
❌ Unmatched in matches: 0
❌ Unmatched in all_matches: 11
2008: 59 matches
2009: 59 matches
2010: 60 matches
2011: 74 matches
2012: 76 matches
2013: 76 matches
2014: 60 matches
2015: 60 matches
2016: 60 matches
2017: 60 matches
2018: 60 matches
2019: 60 matches
2020: 60 matches
2021: 60 matches
2022: 74 matches
2023: 74 matches
2024: 74 matches


In [3]:
def generate_ipl_scorecard(match_id, deliveries=deliveries, matches=matches):
    match_data = matches[matches['Id'] == match_id].iloc[0]
    match_deliveries = deliveries[deliveries['Match_Id'] == match_id]
    
    team1 = match_data['Team1']
    team2 = match_data['Team2']
    match_no = match_data['Match_No']
    
    # Handle toss decision and batting order
    if match_data['Toss_Decision'] == 'Bat':
        first_batting = match_data['Toss_Winner']
        second_batting = team2 if team1 == match_data['Toss_Winner'] else team1
    else:
        second_batting = match_data['Toss_Winner']
        first_batting = team2 if team1 == match_data['Toss_Winner'] else team1
    
    def generate_innings_data(innings_num, batting_team, bowling_team, is_super_over=False):
        innings_data = match_deliveries[
            (match_deliveries['Inning'] == innings_num) & 
            (match_deliveries['Batting_Team'] == batting_team)
        ].copy()
        
        if innings_data.empty:
            return None
        
        # Get all batsmen who faced at least one ball
        all_batsmen = innings_data['Batter'].unique()
        
        # Get batsmen who were actually dismissed
        dismissed_batsmen_data = innings_data[
            (innings_data['Is_Wicket'] == 1) & 
            (innings_data['Player_Dismissed'].notna())
        ]
        dismissed_batsmen = dismissed_batsmen_data['Player_Dismissed'].unique()
        
        # Get the last ball to see who was at crease
        last_ball = innings_data.iloc[-1]
        batsmen_at_crease = {last_ball['Batter'], last_ball['Non_Striker']}
        
        # FIXED: Use the same proper batting order logic for both regular innings and Super Overs
        batsmen_order = []
        seen_batsmen = set()
        current_batsmen = set()
        
        # Start with the openers from the first ball
        first_ball = innings_data.iloc[0]
        striker = first_ball['Batter']
        non_striker = first_ball['Non_Striker']
        
        # Add openers in correct order (striker first, then non-striker)
        batsmen_order.append(striker)
        seen_batsmen.add(striker)
        current_batsmen.add(striker)
        
        if non_striker not in seen_batsmen:
            batsmen_order.append(non_striker)
            seen_batsmen.add(non_striker)
            current_batsmen.add(non_striker)
        
        # Track wickets to determine when new batsmen come in (for both regular and super over)
        for _, ball in innings_data.iterrows():
            batter = ball['Batter']
            non_striker = ball['Non_Striker']
            
            # Add current batsmen if not seen (handles any missing batsmen)
            if batter not in seen_batsmen:
                batsmen_order.append(batter)
                seen_batsmen.add(batter)
                current_batsmen.add(batter)
            
            if non_striker not in seen_batsmen:
                batsmen_order.append(non_striker)
                seen_batsmen.add(non_striker)
                current_batsmen.add(non_striker)
            
            # Handle wickets - when a batsman gets out, track the new batsman
            if ball['Is_Wicket'] == 1 and pd.notna(ball['Player_Dismissed']):
                dismissed_batsman = ball['Player_Dismissed']
                
                # Remove dismissed batsman from current batsmen
                if dismissed_batsman in current_batsmen:
                    current_batsmen.remove(dismissed_batsman)
                
                # Determine who is the new batsman
                # The new batsman will be at the opposite end of the dismissed batsman
                if ball['Batter'] == dismissed_batsman:
                    new_batsman = ball['Non_Striker']
                else:
                    new_batsman = ball['Batter']
                
                # Add new batsman to order if not already seen
                if new_batsman not in seen_batsmen and pd.notna(new_batsman):
                    batsmen_order.append(new_batsman)
                    seen_batsmen.add(new_batsman)
                    current_batsmen.add(new_batsman)
        
        # Fallback: add any remaining batsmen who faced balls but weren't tracked through wickets
        remaining_batsmen = set(all_batsmen) - seen_batsmen
        for batsman in remaining_batsmen:
            if pd.notna(batsman):
                # Find first appearance of this batsman and insert in correct position
                batsman_balls = innings_data[innings_data['Batter'] == batsman]
                if not batsman_balls.empty:
                    first_appearance_idx = batsman_balls.index[0]
                    inserted = False
                    
                    # Try to insert based on when they first appeared
                    for i, existing_batsman in enumerate(batsmen_order):
                        existing_balls = innings_data[innings_data['Batter'] == existing_batsman]
                        if not existing_balls.empty:
                            existing_first_idx = existing_balls.index[0]
                            if first_appearance_idx < existing_first_idx:
                                batsmen_order.insert(i, batsman)
                                inserted = True
                                break
                    
                    if not inserted:
                        batsmen_order.append(batsman)
                    seen_batsmen.add(batsman)
        
        # Batsman statistics with proper order - FIXED BALLS COUNTING
        batsman_data = []
        total_wickets = innings_data['Is_Wicket'].sum()
        
        for batsman in batsmen_order:
            # FIXED: Count balls faced correctly - exclude wides and no-balls from ball count
            batsman_balls_data = innings_data[
                (innings_data['Batter'] == batsman) & 
                (~innings_data['Extras_Type'].isin(['Wides', 'Noballs']))
            ]
            
            runs = batsman_balls_data['Batsman_Runs'].sum()
            balls = len(batsman_balls_data)  # Only count valid balls faced
            strike_rate = round((runs / balls * 100), 2) if balls > 0 else 0
            
            # Check if batsman was dismissed
            was_dismissed = batsman in dismissed_batsmen
            
            # Check if batsman was at crease at the end
            was_at_crease_at_end = batsman in batsmen_at_crease
            
            # Determine dismissal text
            if was_dismissed:
                # Get dismissal details
                dismissal_data = innings_data[
                    (innings_data['Player_Dismissed'] == batsman) & 
                    (innings_data['Is_Wicket'] == 1)
                ].iloc[0]
                
                bowler = dismissal_data['Bowler']
                dismissal = dismissal_data['Dismissal_Kind']
                fielder = dismissal_data['Fielder'] if pd.notna(dismissal_data['Fielder']) else ""
                
                # Handle all dismissal types properly
                if dismissal == 'Caught':
                    dismissal_text = f"c {fielder} b {bowler}"
                elif dismissal == 'Bowled':
                    dismissal_text = f"b {bowler}"
                elif dismissal == 'Lbw':
                    dismissal_text = f"lbw b {bowler}"
                elif dismissal == 'Run Out':
                    # Run out is NOT a bowler's wicket - no bowler mentioned
                    dismissal_text = f"run out ({fielder})" if fielder else "run out"
                elif dismissal == 'Stumped':
                    dismissal_text = f"st {fielder} b {bowler}"
                elif dismissal == 'Caught And Bowled':
                    dismissal_text = f"c & b {bowler}"
                elif dismissal == 'Hit Wicket':
                    dismissal_text = f"hit wicket b {bowler}"
                elif dismissal == 'Obstructing The Field':
                    # No bowler for obstructing the field
                    dismissal_text = f"obstructing the field"
                elif dismissal == 'Retired Hurt':
                    # No bowler for retired hurt
                    dismissal_text = f"retired hurt"
                elif dismissal == 'Retired Out':
                    # No bowler for retired out
                    dismissal_text = f"retired out"
                elif dismissal == 'Handled The Ball':
                    # No bowler for handled the ball
                    dismissal_text = f"handled the ball"
                elif dismissal == 'Timed Out':
                    # No bowler for timed out
                    dismissal_text = f"timed out"
                else:
                    # Fallback for any other dismissal types
                    dismissal_text = f"{dismissal}"
            elif was_at_crease_at_end:
                # Only mark as "not out" if they were actually at the crease when innings ended
                dismissal_text = "not out"
            else:
                # For batsmen who weren't dismissed and weren't at crease at the end
                dismissal_text = ""  # Leave blank
        
            batsman_data.append({
                'Batsman': batsman,
                'Dismissal': dismissal_text,
                'Runs': runs,
                'Balls': balls,
                'SR': strike_rate
            })
        
        batsman_stats = pd.DataFrame(batsman_data)
        
        # Bowler statistics - exclude penalty runs from bowler figures
        bowler_data = []
        bowlers_used = innings_data['Bowler'].unique()
        
        # Maintain bowler order as they appear in the data
        bowler_order = []
        seen_bowlers = set()
        
        for _, ball in innings_data.iterrows():
            bowler = ball['Bowler']
            if bowler not in seen_bowlers:
                bowler_order.append(bowler)
                seen_bowlers.add(bowler)
        
        for bowler in bowler_order:
            bowler_balls = innings_data[innings_data['Bowler'] == bowler]
            
            # Calculate balls bowled (exclude wides and no-balls from ball count)
            valid_balls = bowler_balls[~bowler_balls['Extras_Type'].isin(['Wides', 'Noballs'])]
            balls_bowled = len(valid_balls)
            
            if balls_bowled == 0:
                continue
                
            # Calculate runs conceded (exclude penalty runs)
            runs_conceded = bowler_balls['Batsman_Runs'].sum() + bowler_balls['Extra_Runs'].sum()
            penalty_runs = bowler_balls[bowler_balls['Extras_Type'] == 'Penalty']['Extra_Runs'].sum()
            runs_conceded -= penalty_runs
            
            # Calculate wickets - ONLY count wickets where this bowler was responsible
            # Exclude run outs, obstructing the field, retired, etc.
            bowler_wickets = bowler_balls[
                (bowler_balls['Is_Wicket'] == 1) & 
                (bowler_balls['Player_Dismissed'].notna()) &
                (~bowler_balls['Dismissal_Kind'].isin(['Run Out', 'Obstructing The Field', 'Retired Hurt', 'Retired Out', 'Handled The Ball', 'Timed Out']))
            ]
            wickets = len(bowler_wickets)
            
            # Calculate overs properly (4.1 means 4 overs 1 ball, not 4.1 overs)
            overs = balls_bowled // 6
            balls_in_over = balls_bowled % 6
            overs_display = f"{overs}.{balls_in_over}" if balls_in_over > 0 else f"{overs}"
            
            # Calculate economy rate
            total_overs = overs + (balls_in_over / 6)
            economy = round(runs_conceded / total_overs, 2) if total_overs > 0 else 0
            
            bowler_data.append({
                'Bowler': bowler,
                'Overs': overs_display,
                'Runs': runs_conceded,
                'Wickets': wickets,
                'Economy': economy
            })
        
        bowler_stats = pd.DataFrame(bowler_data)
        
        # Calculate match totals
        extras_breakdown = innings_data.groupby('Extras_Type')['Extra_Runs'].sum()
        extras = extras_breakdown.sum()
        
        # Detailed extras breakdown
        extras_detail = []
        for extra_type in ['Wides', 'Noballs', 'Byes', 'Legbyes', 'Penalty']:
            if extra_type in extras_breakdown and extras_breakdown[extra_type] > 0:
                extras_detail.append(f"{extras_breakdown[extra_type]} {extra_type.lower()}")
        
        total_runs = innings_data['Total_Runs'].sum()
        total_wickets = innings_data['Is_Wicket'].sum()
        
        # Calculate valid balls for total overs (exclude wides and no-balls)
        valid_balls_total = innings_data[~innings_data['Extras_Type'].isin(['Wides', 'Noballs'])]
        total_balls_bowled = len(valid_balls_total)
        total_overs = total_balls_bowled // 6
        total_balls = total_balls_bowled % 6
        
        # Format overs properly (4.1 means 4 overs 1 ball)
        total_overs_display = f"{total_overs}.{total_balls}" if total_balls > 0 else f"{total_overs}"
        
        return {
            'batting_team': batting_team,
            'bowling_team': bowling_team,
            'batsmen': batsman_stats,
            'bowlers': bowler_stats,
            'extras': extras,
            'extras_detail': ', '.join(extras_detail),
            'total_runs': total_runs,
            'total_wickets': total_wickets,
            'total_overs': total_overs_display,
            'total_balls_bowled': total_balls_bowled,
            'is_super_over': is_super_over
        }
    
    # Generate regular innings data
    innings1 = generate_innings_data(1, first_batting, second_batting)
    innings2 = generate_innings_data(2, second_batting, first_batting)
    
    # Generate Super Over innings data - FIXED: Properly identify Super Over teams
    super_over_innings = []
    super_over_numbers = [3, 4, 5, 6]  # Possible super over inning numbers
    
    # Get unique Super Over innings
    so_innings_found = match_deliveries[match_deliveries['Inning'].isin(super_over_numbers)]['Inning'].unique()
    
    for so_inning in sorted(so_innings_found):
        # Find which teams played in this super over
        so_data = match_deliveries[match_deliveries['Inning'] == so_inning]
        if not so_data.empty:
            # Get the batting team for this super over
            so_batting_team = so_data['Batting_Team'].iloc[0]
            so_bowling_team = so_data['Bowling_Team'].iloc[0]
            
            super_over_innings.append({
                'inning_num': so_inning,
                'data': generate_innings_data(so_inning, so_batting_team, so_bowling_team, is_super_over=True)
            })
    
    # Group Super Overs into pairs (Super Over 1, Super Over 2, etc.)
    super_over_pairs = []
    for i in range(0, len(super_over_innings), 2):
        pair = super_over_innings[i:i+2]
        super_over_pairs.append(pair)
    
    # Handle match result with all edge cases including Super Over
    def get_match_result():
        result = match_data['Result']
        winner = match_data['Winner']
        super_over = match_data['Super_Over']
        
        if pd.isna(result) or result == 'No Result':
            return "MATCH ABANDONED - NO RESULT", None, None
        
        if result == 'Tie':
            if super_over == 'Y':
                # For Super Over matches, show the actual winner and margin
                if pd.notna(winner):
                    # Try to determine Super Over margin from the last super over pair
                    if super_over_pairs:
                        last_pair = super_over_pairs[-1]
                        if len(last_pair) == 2:
                            so1 = last_pair[0]['data']
                            so2 = last_pair[1]['data']
                            if so1 and so2:
                                if so1['total_runs'] > so2['total_runs']:
                                    margin = so1['total_runs'] - so2['total_runs']
                                    return f"{winner.upper()} WIN BY {margin} RUNS (Super Over)", winner, "Super Over"
                                else:
                                    margin = 2 - so2['total_wickets']  # Assuming 2 wickets max in super over
                                    return f"{winner.upper()} WIN BY {margin} WICKETS (Super Over)", winner, "Super Over"
                    
                    # Fallback if we can't determine margin
                    return f"{winner.upper()} WIN (Super Over)", winner, "Super Over"
                else:
                    return "MATCH TIED - DECIDED BY SUPER OVER", None, "Super Over"
            else:
                return "MATCH TIED", None, None
        
        # Handle D/L method
        method_note = " (D/L method)" if match_data['Method'] == 'D/L' else ""
        
        # Handle target runs and overs for D/L
        target_note = ""
        if pd.notna(match_data['Target_Runs']) and pd.notna(match_data['Target_Overs']):
            target_runs = int(match_data['Target_Runs'])
            target_overs = int(match_data['Target_Overs']) if match_data['Target_Overs'] % 1 == 0 else match_data['Target_Overs']
            target_note = f" (Target: {target_runs} in {target_overs} overs)"
        
        if result == 'Runs':
            margin = int(match_data['Result_Margin'])
            target_display = f"<br><div style='text-align: center;'>{target_note}</div>" if target_note else ""
            return f"{winner.upper()} WIN BY {margin} RUNS{method_note}{target_display}", winner, None
        elif result == 'Wickets':
            margin = int(match_data['Result_Margin'])
            target_display = f"<br><div style='text-align: center;'>{target_note}</div>" if target_note else ""
            return f"{winner.upper()} WIN BY {margin} WICKETS{method_note}{target_display}", winner, None
        
        return "MATCH COMPLETED", winner, None
    
    result_text, winner, result_type = get_match_result()
    
    # Handle match type display
    match_type = match_data['Match_Type']
    match_type_display = {
        'League': 'LEAGUE MATCH',
        'Semi Final': 'SEMI FINAL',
        'Final': 'FINAL',
        '3Rd Place Play-Off': '3RD PLACE PLAY-OFF',
        'Qualifier 1': 'QUALIFIER 1',
        'Elimination Final': 'ELIMINATION FINAL',
        'Qualifier 2': 'QUALIFIER 2',
        'Eliminator': 'ELIMINATOR'
    }.get(match_type, match_type.upper())
    
    # CSS Styling with super over styling
    css_style = """
    <style>
    .ipl-scorecard {
        font-family: 'Arial', sans-serif;
        max-width: 1000px;
        margin: 20px auto;
        background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%);
        border-radius: 15px;
        box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        overflow: hidden;
    }
    
    .final-match {
        background: linear-gradient(135deg, #8B0000 0%, #FFD700 100%) !important;
    }
    
    .playoff-match {
        background: linear-gradient(135deg, #4B0082 0%, #9370DB 100%) !important;
    }
    
    .super-over-match {
        background: linear-gradient(135deg, #FF4500 0%, #FFA500 100%) !important;
    }
    
    .match-header {
        background: linear-gradient(45deg, #004ba0, #1976d2);
        color: white;
        padding: 20px;
        text-align: center;
        border-bottom: 3px solid #ffd700;
    }
    
    .final-header {
        background: linear-gradient(45deg, #B22222, #DC143C) !important;
    }
    
    .playoff-header {
        background: linear-gradient(45deg, #4B0082, #8A2BE2) !important;
    }
    
    .super-over-header {
        background: linear-gradient(45deg, #FF4500, #FF8C00) !important;
    }
    
    .match-title {
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 5px;
        text-transform: uppercase;
        letter-spacing: 1px;
    }
    
    .match-subtitle {
        font-size: 16px;
        opacity: 0.9;
        margin-bottom: 5px;
    }
    
    .match-type {
        font-size: 14px;
        background: rgba(255,255,255,0.2);
        padding: 5px 10px;
        border-radius: 15px;
        display: inline-block;
        margin-top: 5px;
    }
    
    .super-over-badge {
        background: linear-gradient(45deg, #FF4500, #FF8C00);
        color: white;
        padding: 5px 10px;
        border-radius: 15px;
        font-weight: bold;
        margin-left: 10px;
    }
    
    .innings-container {
        display: flex;
        flex-wrap: wrap;
        gap: 20px;
        padding: 20px;
    }
    
    .innings-card {
        flex: 1;
        min-width: 450px;
        background: white;
        border-radius: 10px;
        box-shadow: 0 5px 15px rgba(0,0,0,0.1);
        overflow: hidden;
    }
    
    .super-over-card {
        border: 3px solid #FF4500;
        background: linear-gradient(135deg, #FFF8DC 0%, #FFEBCD 100%);
    }
    
    .innings-header {
        background: linear-gradient(45deg, #d32f2f, #f44336);
        color: white;
        padding: 15px;
        font-weight: bold;
        font-size: 18px;
        text-align: center;
    }
    
    .super-over-header-card {
        background: linear-gradient(45deg, #FF4500, #FF8C00) !important;
    }
    
    .team-score {
        background: #f8f9fa;
        padding: 15px;
        text-align: center;
        border-bottom: 2px solid #e9ecef;
    }
    
    .super-over-score {
        background: #FFF8DC !important;
        border-bottom: 2px solid #FF4500;
    }
    
    .score-large {
        font-size: 28px;
        font-weight: bold;
        color: #d32f2f;
    }
    
    .super-over-score-large {
        color: #FF4500 !important;
    }
    
    .score-over {
        font-size: 16px;
        color: #666;
        margin-top: 5px;
    }
    
    .section-title {
        background: #e9ecef;
        padding: 10px 15px;
        font-weight: bold;
        color: #495057;
        border-bottom: 1px solid #dee2e6;
    }
    
    .super-over-section {
        background: #FFEBCD !important;
        border-bottom: 1px solid #FF4500;
    }
    
    .batting-table, .bowling-table {
        width: 100%;
        border-collapse: collapse;
        table-layout: fixed;
    }
    
    .batting-table th, .bowling-table th {
        background: #495057;
        color: white;
        padding: 12px 8px;
        text-align: left;
        font-size: 14px;
    }
    
    .super-over-table th {
        background: #FF4500 !important;
    }
    
    .batting-table td, .bowling-table td {
        padding: 10px 8px;
        border-bottom: 1px solid #e9ecef;
        font-size: 14px;
        word-wrap: break-word;
    }
    
    .batting-table tr:hover, .bowling-table tr:hover {
        background: #f8f9fa;
    }
    
    .not-out {
        color: #28a745;
        font-weight: bold;
    }
    
    .result-section {
        background: linear-gradient(45deg, #388e3c, #4caf50);
        color: white;
        padding: 25px;
        text-align: center;
        border-top: 3px solid #ffd700;
    }
    
    .tie-result {
        background: linear-gradient(45deg, #FF8C00, #FFA500) !important;
    }
    
    .no-result {
        background: linear-gradient(45deg, #696969, #808080) !important;
    }
    
    .super-over-result {
        background: linear-gradient(45deg, #FF4500, #FF8C00) !important;
    }
    
    .result-text {
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 10px;
    }
    
    .mom-text {
        font-size: 18px;
        opacity: 0.9;
        margin-top: 10px;
    }
    
    .extras-total {
        background: #e3f2fd;
        padding: 10px 15px;
        border-top: 1px solid #bbdefb;
        font-weight: bold;
    }
    
    .super-over-extras {
        background: #FFEBCD !important;
        border-top: 1px solid #FF4500;
    }
    
    .extras-detail {
        font-size: 12px;
        color: #666;
        margin-top: 5px;
    }
    
    .dls-note {
        background: #fff3cd;
        color: #856404;
        padding: 10px;
        margin: 10px 0;
        border-radius: 5px;
        border-left: 4px solid #ffc107;
        font-size: 14px;
    }
    
    .target-note {
        background: #e8f5e8;
        color: #2e7d32;
        padding: 10px;
        margin: 10px 0;
        border-radius: 5px;
        border-left: 4px solid #4caf50;
        font-size: 14px;
    }
    
    .super-over-note {
        background: #ffe6cc;
        color: #cc5500;
        padding: 10px;
        margin: 10px 0;
        border-radius: 5px;
        border-left: 4px solid #ff4500;
        font-size: 14px;
        font-weight: bold;
    }
    
    /* Column widths for better formatting */
    .batting-table th:nth-child(1), .batting-table td:nth-child(1) { width: 25%; }
    .batting-table th:nth-child(2), .batting-table td:nth-child(2) { width: 35%; }
    .batting-table th:nth-child(3), .batting-table td:nth-child(3) { width: 10%; }
    .batting-table th:nth-child(4), .batting-table td:nth-child(4) { width: 10%; }
    .batting-table th:nth-child(5), .batting-table td:nth-child(5) { width: 20%; }
    
    .bowling-table th:nth-child(1), .bowling-table td:nth-child(1) { width: 35%; }
    .bowling-table th:nth-child(2), .bowling-table td:nth-child(2) { width: 15%; }
    .bowling-table th:nth-child(3), .bowling-table td:nth-child(3) { width: 15%; }
    .bowling-table th:nth-child(4), .bowling-table td:nth-child(4) { width: 15%; }
    .bowling-table th:nth-child(5), .bowling-table td:nth-child(5) { width: 20%; }
    
    @media (max-width: 768px) {
        .innings-card {
            min-width: 100%;
        }
        .batting-table, .bowling-table {
            font-size: 12px;
        }
    }
    </style>
    """
    
    # Determine special styling for match type
    card_class = "ipl-scorecard"
    header_class = "match-header"
    result_class = "result-section"
    
    if 'Final' in match_type:
        card_class += " final-match"
        header_class += " final-header"
    elif match_type in ['Semi Final', 'Qualifier 1', 'Qualifier 2', 'Eliminator', 'Elimination Final', '3Rd Place Play-Off']:
        card_class += " playoff-match"
        header_class += " playoff-header"
    
    if match_data['Super_Over'] == 'Y':
        card_class += " super-over-match"
        header_class += " super-over-header"
        result_class += " super-over-result"
    elif 'Tie' in result_text:
        result_class += " tie-result"
    elif 'No Result' in result_text or 'ABANDONED' in result_text:
        result_class += " no-result"
    
    # Generate HTML content
    html_content = f"""
    {css_style}
    <div class="{card_class}">
        <div class="{header_class}">
            <div class="match-title">TATA IPL {match_data['Season']}</div>
            <div class="match-subtitle">{match_no} • {match_data['Venue']}</div>
            <div class="match-subtitle">{match_data['City']} • {pd.to_datetime(match_data['Date']).strftime('%d %b %Y')}</div>
            <div class="match-type">{match_type_display}</div>
            {f'<span class="super-over-badge">SUPER OVER MATCH</span>' if match_data['Super_Over'] == 'Y' else ''}
        </div>
    """
    
    # Add DLS note if applicable
    if match_data['Method'] == 'D/L':
        html_content += f"""
        <div class="dls-note" style="text-align: center;">
            <strong>Duckworth-Lewis-Stern Method Applied:</strong> Target adjusted due to weather interruptions
        </div>
        """
    
    # Add target note if applicable
    # if pd.notna(match_data['Target_Runs']) and pd.notna(match_data['Target_Overs']):
    #     target_runs = match_data['Target_Runs']
    #     target_overs = match_data['Target_Overs']
    #     html_content += f"""
    #     <div class="target-note">
    #         <strong>Revised Target:</strong> {target_runs} runs in {target_overs} overs
    #     </div>
    #     """
    
    # Add Super Over note if applicable
    if match_data['Super_Over'] == 'Y':
        html_content += f"""
        <div class="super-over-note" style="text-align: center;">
            <strong>SUPER OVER MATCH:</strong> Regular match tied, decided by Super Over
        </div>
        """
    
    html_content += """
        <div class="innings-container">
    """
    
    # Function to render innings
    def render_innings(innings, innings_title, is_super_over=False):
        if not innings:
            return f"""
            <div class="innings-card">
                <div class="innings-header">{innings_title} DATA NOT AVAILABLE</div>
            </div>
            """
        
        card_class = "innings-card super-over-card" if is_super_over else "innings-card"
        header_class = "innings-header super-over-header-card" if is_super_over else "innings-header"
        score_class = "team-score super-over-score" if is_super_over else "team-score"
        score_large_class = "score-large super-over-score-large" if is_super_over else "score-large"
        section_class = "section-title super-over-section" if is_super_over else "section-title"
        extras_class = "extras-total super-over-extras" if is_super_over else "extras-total"
        table_class = "super-over-table" if is_super_over else ""
        
        html = f"""
            <div class="{card_class}">
                <div class="{header_class}">{innings_title}</div>
                <div class="{score_class}">
                    <div class="{score_large_class}">{innings['total_runs']}-{innings['total_wickets']}</div>
                    <div class="score-over">OVERS {innings['total_overs']}</div>
                </div>
                
                <div class="{section_class}">BATTING</div>
                <table class="batting-table {table_class}">
                    <thead>
                        <tr>
                            <th>Batsman</th>
                            <th>Dismissal</th>
                            <th>R</th>
                            <th>B</th>
                            <th>SR</th>
                        </tr>
                    </thead>
                    <tbody>
        """
        
        # Add batsmen rows
        for _, batsman in innings['batsmen'].iterrows():
            not_out_class = "not-out" if "not out" in batsman['Dismissal'].lower() else ""
            html += f"""
                        <tr>
                            <td><strong>{batsman['Batsman']}</strong></td>
                            <td class="{not_out_class}">{batsman['Dismissal']}</td>
                            <td><strong>{batsman['Runs']}</strong></td>
                            <td>{batsman['Balls']}</td>
                            <td>{batsman['SR']}</td>
                        </tr>
            """
        
        html += f"""
                    </tbody>
                </table>
                <div class="{extras_class}">
                    Extras: {innings['extras']}
                    <div class="extras-detail">{innings['extras_detail']}</div>
                </div>
                
                <div class="{section_class}">BOWLING</div>
                <table class="bowling-table {table_class}">
                    <thead>
                        <tr>
                            <th>Bowler</th>
                            <th>O</th>
                            <th>R</th>
                            <th>W</th>
                            <th>Econ</th>
                        </tr>
                    </thead>
                    <tbody>
        """
        
        # Add bowlers rows
        if not innings['bowlers'].empty:
            for _, bowler in innings['bowlers'].iterrows():
                html += f"""
                        <tr>
                            <td><strong>{bowler['Bowler']}</strong></td>
                            <td>{bowler['Overs']}</td>
                            <td>{bowler['Runs']}</td>
                            <td><strong>{bowler['Wickets']}</strong></td>
                            <td>{bowler['Economy']}</td>
                        </tr>
                """
        else:
            html += """
                        <tr>
                            <td colspan="5" style="text-align: center; padding: 20px;">No bowling data available</td>
                        </tr>
            """
        
        html += """
                    </tbody>
                </table>
            </div>
        """
        return html
    
    # Render regular innings
    if innings1:
        html_content += render_innings(innings1, first_batting.upper())
    if innings2:
        html_content += render_innings(innings2, second_batting.upper())
    
    # Render Super Over pairs
    for i, pair in enumerate(super_over_pairs):
        so_number = i + 1
        for so_data in pair:
            if so_data['data']:
                innings_title = f"SUPER OVER {so_number} - {so_data['data']['batting_team'].upper()}"
                html_content += render_innings(so_data['data'], innings_title, is_super_over=True)
    
    html_content += f"""
        </div>
        
        <div class="{result_class}">
            <div class="result-text">{result_text}</div>
    """
    
    # Add toss information
    html_content += f"""
            <div class="match-subtitle">
                Toss: {match_data['Toss_Winner']} chose to {match_data['Toss_Decision'].lower()}
            </div>
    """
    
    # Add Man of the Match if available
    if pd.notna(match_data['Player_Of_Match']):
        html_content += f'<div class="mom-text">PLAYER OF THE MATCH: {match_data["Player_Of_Match"]}</div>'
    
    # Add super over note if applicable
    # if match_data['Super_Over'] == 'Y':
    #     html_content += '<div class="mom-text">• Match decided by Super Over</div>'
    
    html_content += """
        </div>
    </div>
    """
    
    return HTML(html_content)

In [4]:
scorecard_html = generate_ipl_scorecard(335982)
display(scorecard_html)


Batsman,Dismissal,R,B,SR
SC Ganguly,c JH Kallis b Z Khan,10,12,83.33
BB McCullum,not out,158,73,216.44
RT Ponting,c P Kumar b JH Kallis,20,20,100.0
DJ Hussey,c CL White b AA Noffke,12,12,100.0
Mohammad Hafeez,not out,5,3,166.67
Bowler,O,R,W,Econ
P Kumar,4,41,0,10.25
Z Khan,4,38,1,9.5
AA Noffke,4,41,1,10.25
JH Kallis,4,52,1,13.0


In [5]:
def generate_ipl_match_summary(match_id, deliveries=deliveries, matches=matches):
    """
    Generate a compact IPL match summary with dark mode styling
    """
    # Get match data
    match_data = matches[matches['Id'] == match_id].iloc[0]
    match_deliveries = deliveries[deliveries['Match_Id'] == match_id]
    
    team1 = match_data['Team1']
    team2 = match_data['Team2']
    match_no = match_data['Match_No']
    
    # Handle toss decision and batting order
    if match_data['Toss_Decision'] == 'Bat':
        first_batting = match_data['Toss_Winner']
        second_batting = team2 if team1 == match_data['Toss_Winner'] else team1
    else:
        second_batting = match_data['Toss_Winner']
        first_batting = team2 if team1 == match_data['Toss_Winner'] else team1
    
    def get_compact_innings_data(innings_num, batting_team, bowling_team):
        """
        Extract compact innings data with top performers
        """
        innings_data = match_deliveries[
            (match_deliveries['Inning'] == innings_num) & 
            (match_deliveries['Batting_Team'] == batting_team)
        ].copy()
        
        if innings_data.empty:
            return None
        
        # Get batsmen data (without dismissal info)
        batsmen_data = []
        all_batsmen = innings_data['Batter'].unique()
        
        for batsman in all_batsmen:
            # Count valid balls faced (excluding wides and no-balls)
            batsman_balls = innings_data[
                (innings_data['Batter'] == batsman) & 
                (~innings_data['Extras_Type'].isin(['Wides', 'Noballs']))
            ]
            runs = batsman_balls['Batsman_Runs'].sum()
            balls = len(batsman_balls)
            
            batsmen_data.append({
                'name': batsman,
                'runs': runs,
                'balls': balls,
                'strike_rate': round((runs / balls * 100), 1) if balls > 0 else 0
            })
        
        # Get top 4 batsmen by runs
        top_batsmen = sorted(batsmen_data, key=lambda x: x['runs'], reverse=True)[:4]
        
        # Get bowler data
        bowler_data = []
        bowlers_used = innings_data['Bowler'].unique()
        
        for bowler in bowlers_used:
            bowler_balls = innings_data[innings_data['Bowler'] == bowler]
            valid_balls = bowler_balls[~bowler_balls['Extras_Type'].isin(['Wides', 'Noballs'])]
            balls_bowled = len(valid_balls)
            
            if balls_bowled == 0:
                continue
                
            runs_conceded = bowler_balls['Total_Runs'].sum()
            
            # Calculate wickets (excluding run outs and retirements)
            bowler_wickets = bowler_balls[
                (bowler_balls['Is_Wicket'] == 1) & 
                (bowler_balls['Player_Dismissed'].notna()) &
                (~bowler_balls['Dismissal_Kind'].isin(['Run Out', 'Obstructing The Field', 'Retired Hurt', 'Retired Out']))
            ]
            wickets = len(bowler_wickets)
            
            # Format overs
            overs = balls_bowled // 6
            balls_in_over = balls_bowled % 6
            overs_display = f"{overs}.{balls_in_over}" if balls_in_over > 0 else f"{overs}"
            
            # Calculate economy rate
            total_overs = overs + (balls_in_over / 6)
            economy = round(runs_conceded / total_overs, 1) if total_overs > 0 else 0
            
            bowler_data.append({
                'name': bowler,
                'wickets': wickets,
                'runs': runs_conceded,
                'overs': overs_display,
                'economy': economy
            })
        
        # Sort bowlers: first by wickets (descending), then by economy (ascending)
        # Show wicket-takers first, then best economy
        wicket_takers = [b for b in bowler_data if b['wickets'] > 0]
        non_wicket_takers = [b for b in bowler_data if b['wickets'] == 0]
        
        # Sort wicket takers by wickets (desc) then economy (asc)
        wicket_takers_sorted = sorted(wicket_takers, key=lambda x: (-x['wickets'], x['economy']))
        
        # Sort non-wicket takers by economy (asc)
        non_wicket_takers_sorted = sorted(non_wicket_takers, key=lambda x: x['economy'])
        
        # Combine lists - wicket takers first, then best economy
        top_bowlers = (wicket_takers_sorted + non_wicket_takers_sorted)[:4]
        
        # Calculate innings totals
        total_runs = innings_data['Total_Runs'].sum()
        total_wickets = innings_data['Is_Wicket'].sum()
        
        # Calculate overs bowled
        valid_balls_total = innings_data[~innings_data['Extras_Type'].isin(['Wides', 'Noballs'])]
        total_balls_bowled = len(valid_balls_total)
        total_overs = total_balls_bowled // 6
        total_balls = total_balls_bowled % 6
        total_overs_display = f"{total_overs}.{total_balls}" if total_balls > 0 else f"{total_overs}"
        
        # Calculate extras
        extras = innings_data['Extra_Runs'].sum()
        
        return {
            'batting_team': batting_team,
            'bowling_team': bowling_team,
            'total': f"{total_runs}-{total_wickets}",
            'overs': total_overs_display,
            'top_batsmen': top_batsmen,
            'top_bowlers': top_bowlers,
            'extras': extras
        }
    
    # Get innings data
    innings1 = get_compact_innings_data(1, first_batting, second_batting)
    innings2 = get_compact_innings_data(2, second_batting, first_batting)
    
    def get_super_over_summary():
        """
        Get super over summary showing best performers from both teams in each super over
        """
        super_over_numbers = [3, 4, 5, 6]
        so_innings_found = match_deliveries[match_deliveries['Inning'].isin(super_over_numbers)]['Inning'].unique()
        
        super_over_groups = {}
        
        for so_inning in sorted(so_innings_found):
            so_data = match_deliveries[match_deliveries['Inning'] == so_inning]
            if not so_data.empty:
                so_batting_team = so_data['Batting_Team'].iloc[0]
                so_bowling_team = so_data['Bowling_Team'].iloc[0]
                
                # Determine super over pair (Super Over 1, Super Over 2, etc.)
                so_pair_num = (so_inning - 3) // 2 + 1
                
                if so_pair_num not in super_over_groups:
                    super_over_groups[so_pair_num] = {}
                
                # Get best batsman from this super over
                batsmen_runs = so_data.groupby('Batter')['Batsman_Runs'].sum()
                if not batsmen_runs.empty:
                    top_scorer = batsmen_runs.idxmax()
                    top_runs = batsmen_runs.max()
                    
                    # Get balls faced by top scorer
                    scorer_balls = so_data[
                        (so_data['Batter'] == top_scorer) & 
                        (~so_data['Extras_Type'].isin(['Wides', 'Noballs']))
                    ]
                    balls_faced = len(scorer_balls)
                    
                    # Get best bowler from this super over
                    bowler_data = []
                    for bowler in so_data['Bowler'].unique():
                        bowler_balls = so_data[so_data['Bowler'] == bowler]
                        wickets = len(bowler_balls[
                            (bowler_balls['Is_Wicket'] == 1) & 
                            (bowler_balls['Player_Dismissed'].notna()) &
                            (~bowler_balls['Dismissal_Kind'].isin(['Run Out', 'Obstructing The Field', 'Retired Hurt', 'Retired Out']))
                        ])
                        runs_conceded = bowler_balls['Total_Runs'].sum()
                        valid_balls = bowler_balls[~bowler_balls['Extras_Type'].isin(['Wides', 'Noballs'])]
                        balls_bowled = len(valid_balls)
                        
                        if balls_bowled > 0:
                            overs = balls_bowled // 6
                            balls_in_over = balls_bowled % 6
                            overs_display = f"{overs}.{balls_in_over}" if balls_in_over > 0 else f"{overs}"
                            bowler_data.append({
                                'name': bowler,
                                'wickets': wickets,
                                'runs': runs_conceded,
                                'overs': overs_display
                            })
                    
                    # Sort bowlers by wickets then runs conceded
                    if bowler_data:
                        best_bowler = sorted(bowler_data, key=lambda x: (-x['wickets'], x['runs']))[0]
                    else:
                        best_bowler = {'name': 'No bowler', 'wickets': 0, 'runs': 0, 'overs': '0'}
                    
                    super_over_groups[so_pair_num][so_batting_team] = {
                        'top_scorer': {
                            'name': top_scorer,
                            'runs': top_runs,
                            'balls': balls_faced
                        },
                        'best_bowler': best_bowler,
                        'total_runs': so_data['Total_Runs'].sum(),
                        'total_wickets': so_data['Is_Wicket'].sum()
                    }
        
        return super_over_groups
    
    # Get super over summary
    super_over_groups = get_super_over_summary()
    
    def get_match_result():
        """
        Determine match result with all edge cases
        """
        result = match_data['Result']
        winner = match_data['Winner']
        super_over = match_data['Super_Over']
        
        if pd.isna(result) or result == 'No Result':
            return "MATCH ABANDONED - NO RESULT", None
        
        if result == 'Tie':
            if super_over == 'Y' and pd.notna(winner):
                # Check if we have multiple super overs
                if len(super_over_groups) > 1:
                    return f"{winner.upper()} WIN (After {len(super_over_groups)} Super Overs)", winner
                else:
                    return f"{winner.upper()} WIN (Super Over)", winner
            return "MATCH TIED", None
        
        # Add D/L method note if applicable
        method_note = " (D/L method)" if match_data['Method'] == 'D/L' else ""
        
        if result == 'Runs':
            margin = int(match_data['Result_Margin'])
            return f"{winner.upper()} WIN BY {margin} RUNS{method_note}", winner
        elif result == 'Wickets':
            margin = int(match_data['Result_Margin'])
            return f"{winner.upper()} WIN BY {margin} WICKETS{method_note}", winner
        
        return "MATCH COMPLETED", winner
    
    result_text, winner = get_match_result()
    
    # Match type display configuration
    match_type = match_data['Match_Type']
    match_type_display = {
        'League': 'LEAGUE MATCH',
        'Semi Final': 'SEMI FINAL', 
        'Final': 'FINAL',
        'Qualifier 1': 'QUALIFIER 1',
        'Qualifier 2': 'QUALIFIER 2',
        'Eliminator': 'ELIMINATOR',
        'Elimination Final': 'ELIMINATION FINAL',
        '3Rd Place Play-Off': '3RD PLACE PLAY-OFF'
    }.get(match_type, match_type.upper())
    
    # Dark Mode CSS Styling
    css_style = """
    <style>
    .ipl-compact-summary {
        font-family: 'Segoe UI', 'Arial', sans-serif;
        max-width: 800px;
        margin: 20px auto;
        background: linear-gradient(135deg, #0f172a 0%, #1e293b 100%);
        border-radius: 12px;
        box-shadow: 0 8px 32px rgba(0,0,0,0.4);
        overflow: hidden;
        color: #e2e8f0;
        border: 1px solid #334155;
    }
    
    .final-match {
        background: linear-gradient(135deg, #7f1d1d 0%, #991b1b 100%) !important;
        border-color: #dc2626;
    }
    
    .playoff-match {
        background: linear-gradient(135deg, #3730a3 0%, #4f46e5 100%) !important;
        border-color: #6366f1;
    }
    
    .super-over-match {
        background: linear-gradient(135deg, #7c2d12 0%, #ea580c 100%) !important;
        border-color: #f97316;
    }
    
    .match-header {
        background: linear-gradient(135deg, rgba(15, 23, 42, 0.9) 0%, rgba(30, 41, 59, 0.9) 100%);
        padding: 20px;
        text-align: center;
        border-bottom: 2px solid #f59e0b;
        backdrop-filter: blur(10px);
    }
    
    .final-header {
        background: linear-gradient(135deg, rgba(127, 29, 29, 0.9) 0%, rgba(153, 27, 27, 0.9) 100%) !important;
        border-bottom-color: #fbbf24;
    }
    
    .playoff-header {
        background: linear-gradient(135deg, rgba(55, 48, 163, 0.9) 0%, rgba(79, 70, 229, 0.9) 100%) !important;
        border-bottom-color: #a5b4fc;
    }
    
    .super-over-header {
        background: linear-gradient(135deg, rgba(124, 45, 18, 0.9) 0%, rgba(234, 88, 12, 0.9) 100%) !important;
        border-bottom-color: #fdba74;
    }
    
    .match-title {
        font-size: 20px;
        font-weight: 700;
        margin-bottom: 4px;
        text-transform: uppercase;
        color: #f8fafc;
        letter-spacing: 0.5px;
    }
    
    .match-subtitle {
        font-size: 13px;
        color: #cbd5e1;
        margin-bottom: 3px;
        font-weight: 500;
    }
    
    .match-type-badge {
        display: inline-block;
        background: linear-gradient(135deg, #475569 0%, #64748b 100%);
        color: #f1f5f9;
        padding: 6px 14px;
        border-radius: 20px;
        font-size: 11px;
        font-weight: 600;
        margin-top: 8px;
        text-transform: uppercase;
        letter-spacing: 0.5px;
        border: 1px solid #475569;
    }
    
    .super-over-badge {
        background: linear-gradient(135deg, #ea580c 0%, #f97316 100%) !important;
        border-color: #ea580c;
        margin-left: 8px;
    }
    
    .match-info-bar {
        background: rgba(30, 41, 59, 0.8);
        padding: 12px 20px;
        font-size: 12px;
        text-align: center;
        color: #94a3b8;
        border-bottom: 1px solid #334155;
        font-weight: 500;
    }
    
    .teams-container {
        display: flex;
        gap: 16px;
        padding: 20px;
    }
    
    .team-card {
        flex: 1;
        background: rgba(30, 41, 59, 0.6);
        border-radius: 10px;
        padding: 18px;
        backdrop-filter: blur(8px);
        border: 1px solid #374151;
        transition: transform 0.2s ease, border-color 0.2s ease;
    }
    
    .team-card:hover {
        transform: translateY(-2px);
        border-color: #4b5563;
    }
    
    .team-header {
        text-align: center;
        margin-bottom: 16px;
        padding-bottom: 12px;
        border-bottom: 2px solid rgba(245, 158, 11, 0.3);
    }
    
    .team-name {
        font-size: 15px;
        font-weight: 700;
        color: #f59e0b;
        margin-bottom: 6px;
        text-transform: uppercase;
        letter-spacing: 0.5px;
    }
    
    .team-score {
        font-size: 26px;
        font-weight: 800;
        margin-bottom: 4px;
        color: #f8fafc;
        text-shadow: 0 2px 4px rgba(0,0,0,0.3);
    }
    
    .team-overs {
        font-size: 13px;
        color: #94a3b8;
        font-weight: 500;
    }
    
    .players-section {
        margin-top: 16px;
    }
    
    .section-title {
        font-size: 11px;
        font-weight: 700;
        color: #f59e0b;
        margin-bottom: 10px;
        padding-bottom: 6px;
        border-bottom: 1px solid #4b5563;
        text-transform: uppercase;
        letter-spacing: 0.5px;
    }
    
    .player-row {
        display: flex;
        justify-content: space-between;
        align-items: center;
        font-size: 12px;
        margin-bottom: 8px;
        padding: 6px 0;
        border-bottom: 1px solid #374151;
        transition: background-color 0.2s ease;
    }
    
    .player-row:hover {
        background: rgba(55, 65, 81, 0.3);
        border-radius: 4px;
        padding: 6px 8px;
        margin: 0 -8px 8px -8px;
    }
    
    .player-name {
        flex: 2;
        font-weight: 600;
        color: #e2e8f0;
    }
    
    .player-stats {
        flex: 1;
        text-align: right;
        font-weight: 700;
        color: #f8fafc;
    }
    
    .bowler-row {
        display: flex;
        justify-content: space-between;
        align-items: center;
        font-size: 12px;
        margin-bottom: 8px;
        padding: 6px 0;
        border-bottom: 1px solid #374151;
        transition: background-color 0.2s ease;
    }
    
    .bowler-row:hover {
        background: rgba(55, 65, 81, 0.3);
        border-radius: 4px;
        padding: 6px 8px;
        margin: 0 -8px 8px -8px;
    }
    
    .bowler-name {
        flex: 2;
        font-weight: 600;
        color: #e2e8f0;
    }
    
    .bowler-stats {
        flex: 2;
        text-align: right;
        font-weight: 700;
        color: #f8fafc;
        font-size: 11px;
    }
    
    .extras-row {
        font-size: 11px;
        color: #9ca3af;
        margin-top: 10px;
        padding-top: 10px;
        border-top: 1px solid #4b5563;
        font-weight: 500;
    }
    
    .super-over-container {
        display: flex;
        gap: 16px;
        padding: 0 20px 20px 20px;
    }
    
    .super-over-title {
        font-size: 14px;
        font-weight: 700;
        color: #fdba74;
        margin: 0 20px 12px 20px;
        padding-top: 20px;
        text-transform: uppercase;
        letter-spacing: 0.5px;
        text-align: center;
        border-top: 2px solid rgba(234, 88, 12, 0.3);
    }
    
    .super-over-card {
        flex: 1;
        background: rgba(124, 45, 18, 0.3);
        border-radius: 10px;
        padding: 16px;
        border: 2px solid #ea580c;
        backdrop-filter: blur(8px);
    }
    
    .so-team-header {
        text-align: center;
        margin-bottom: 12px;
        padding-bottom: 8px;
        border-bottom: 1px solid rgba(253, 186, 116, 0.3);
    }
    
    .so-team-name {
        font-size: 14px;
        font-weight: 700;
        color: #fdba74;
        margin-bottom: 4px;
        text-transform: uppercase;
    }
    
    .so-team-score {
        font-size: 18px;
        font-weight: 800;
        color: #f8fafc;
    }
    
    .so-players-section {
        margin-top: 12px;
    }
    
    .so-section-title {
        font-size: 10px;
        font-weight: 700;
        color: #fdba74;
        margin-bottom: 8px;
        padding-bottom: 4px;
        border-bottom: 1px solid rgba(234, 88, 12, 0.3);
        text-transform: uppercase;
    }
    
    .so-player-row {
        display: flex;
        justify-content: space-between;
        align-items: center;
        font-size: 11px;
        margin-bottom: 6px;
        padding: 4px 0;
    }
    
    .so-player-name {
        flex: 2;
        font-weight: 600;
        color: #e2e8f0;
    }
    
    .so-player-stats {
        flex: 1;
        text-align: right;
        font-weight: 700;
        color: #f8fafc;
    }
    
    .result-section {
        background: linear-gradient(135deg, rgba(15, 23, 42, 0.9) 0%, rgba(30, 41, 59, 0.9) 100%);
        padding: 24px;
        text-align: center;
        border-top: 2px solid #f59e0b;
        backdrop-filter: blur(10px);
    }
    
    .result-text {
        font-size: 22px;
        font-weight: 800;
        color: #f59e0b;
        margin-bottom: 12px;
        text-shadow: 0 2px 4px rgba(0,0,0,0.3);
        text-transform: uppercase;
        letter-spacing: 0.5px;
    }
    
    .match-details {
        font-size: 12px;
        color: #cbd5e1;
        line-height: 1.6;
        font-weight: 500;
    }
    
    .special-notes {
        background: rgba(55, 65, 81, 0.6);
        padding: 12px 16px;
        margin: 12px 20px;
        border-radius: 8px;
        font-size: 11px;
        text-align: center;
        border-left: 4px solid;
        backdrop-filter: blur(8px);
        font-weight: 600;
    }
    
    .dls-note {
        background: rgba(180, 83, 9, 0.2) !important;
        border-left-color: #f59e0b;
        color: #fbbf24;
    }
    
    .super-over-note {
        background: rgba(194, 65, 12, 0.2) !important;
        border-left-color: #ea580c;
        color: #fdba74;
    }
    
    .mom-highlight {
        color: #f59e0b;
        font-weight: 700;
    }
    
    .target-info {
        color: #60a5fa;
        font-weight: 600;
    }
    
    .wicket-taker {
        color: #10b981 !important;
    }
    
    .economy-bowler {
        color: #60a5fa !important;
    }
    
    @media (max-width: 768px) {
        .teams-container {
            flex-direction: column;
            gap: 12px;
            padding: 16px;
        }
        
        .team-card {
            min-width: 100%;
        }
        
        .super-over-container {
            flex-direction: column;
            gap: 12px;
            padding: 0 16px 16px 16px;
        }
        
        .match-title {
            font-size: 18px;
        }
        
        .team-score {
            font-size: 24px;
        }
    }
    
    /* Animation for result */
    @keyframes pulse-gold {
        0%, 100% { color: #f59e0b; }
        50% { color: #fbbf24; }
    }
    
    .result-text {
        animation: pulse-gold 2s ease-in-out infinite;
    }
    </style>
    """
    
    # Determine special styling based on match type
    card_class = "ipl-compact-summary"
    header_class = "match-header"
    
    if 'Final' in match_type:
        card_class += " final-match"
        header_class += " final-header"
    elif match_type in ['Semi Final', 'Qualifier 1', 'Qualifier 2', 'Eliminator', 'Elimination Final']:
        card_class += " playoff-match"
        header_class += " playoff-header"
    
    if match_data['Super_Over'] == 'Y':
        card_class += " super-over-match"
        header_class += " super-over-header"
    
    # Generate HTML content
    html_content = f"""
    {css_style}
    <div class="{card_class}">
        <div class="{header_class}">
            <div class="match-title">TATA IPL {match_data['Season']}</div>
            <div class="match-subtitle">{match_no} • {match_data['Venue']}</div>
            <div class="match-subtitle">{pd.to_datetime(match_data['Date']).strftime('%d %b %Y')}</div>
            <div class="match-type-badge">{match_type_display}</div>
            {f'<div class="match-type-badge super-over-badge">SUPER OVER</div>' if match_data['Super_Over'] == 'Y' else ''}
        </div>
        
        <div class="match-info-bar">
            Toss: {match_data['Toss_Winner']} chose to {match_data['Toss_Decision'].lower()}
        </div>
    """
    
    # Add special notes for DLS and Super Over
    if match_data['Method'] == 'D/L':
        html_content += """
        <div class="special-notes dls-note">
            🏏 <strong>Duckworth-Lewis-Stern Method Applied</strong> - Target adjusted due to weather interruptions
        </div>
        """
    
    if match_data['Super_Over'] == 'Y':
        html_content += """
        <div class="special-notes super-over-note">
            ⚡ <strong>SUPER OVER MATCH</strong> - Regular match tied, decided by Super Over
        </div>
        """
    
    html_content += """
        <div class="teams-container">
    """
    
    # Render team cards for both innings
    for innings in [innings1, innings2]:
        if innings:
            html_content += f"""
            <div class="team-card">
                <div class="team-header">
                    <div class="team-name">{innings['batting_team'].upper()}</div>
                    <div class="team-score">{innings['total']}</div>
                    <div class="team-overs">OVERS {innings['overs']}</div>
                </div>
                
                <div class="players-section">
                    <div class="section-title">BATTING</div>
            """
            
            # Add top batsmen (without dismissal info)
            for batsman in innings['top_batsmen']:
                html_content += f"""
                    <div class="player-row">
                        <div class="player-name">{batsman['name']}</div>
                        <div class="player-stats">{batsman['runs']} ({batsman['balls']})</div>
                    </div>
                """
            
            html_content += f"""
                    <div class="extras-row">Extras: {innings['extras']}</div>
                </div>
                
                <div class="players-section">
                    <div class="section-title">BOWLING</div>
            """
            
            # Add top bowlers with proper formatting: "Mohammed Shami 2-30(4)"
            if innings['top_bowlers']:
                for bowler in innings['top_bowlers']:
                    bowler_class = "wicket-taker" if bowler['wickets'] > 0 else "economy-bowler"
                    html_content += f"""
                        <div class="bowler-row">
                            <div class="bowler-name">{bowler['name']}</div>
                            <div class="bowler-stats {bowler_class}">{bowler['wickets']}-{bowler['runs']}({bowler['overs']})</div>
                        </div>
                    """
            else:
                html_content += """
                    <div class="bowler-row">
                        <div class="bowler-name">No bowlers data</div>
                    </div>
                """
            
            html_content += """
                </div>
            </div>
            """
    
    html_content += """
        </div>
    """
    
    # Add Super Over sections for each super over pair
    for so_num, so_data in super_over_groups.items():
        html_content += f"""
        <div class="super-over-title">SUPER OVER {so_num}</div>
        <div class="super-over-container">
        """
        
        # Show both teams that played in this super over
        for team_name, team_data in so_data.items():
            html_content += f"""
            <div class="super-over-card">
                <div class="so-team-header">
                    <div class="so-team-name">{team_name.upper()}</div>
                    <div class="so-team-score">{team_data['total_runs']}-{team_data['total_wickets']}</div>
                </div>
                
                <div class="so-players-section">
                    <div class="so-section-title">BATTING</div>
                    <div class="so-player-row">
                        <div class="so-player-name">{team_data['top_scorer']['name']}</div>
                        <div class="so-player-stats">{team_data['top_scorer']['runs']}({team_data['top_scorer']['balls']})</div>
                    </div>
                </div>
                
                <div class="so-players-section">
                    <div class="so-section-title">BOWLING</div>
                    <div class="so-player-row">
                        <div class="so-player-name">{team_data['best_bowler']['name']}</div>
                        <div class="so-player-stats">{team_data['best_bowler']['wickets']}-{team_data['best_bowler']['runs']}({team_data['best_bowler']['overs']})</div>
                    </div>
                </div>
            </div>
            """
        
        html_content += """
        </div>
        """
    
    html_content += f"""
        <div class="result-section">
            <div class="result-text">{result_text}</div>
            <div class="match-details">
    """
    
    # Add match details
    details = []
    if pd.notna(match_data['Player_Of_Match']):
        details.append(f'Player of the Match: <span class="mom-highlight">{match_data["Player_Of_Match"]}</span>')
    
    # Add target info if D/L method was applied
    if pd.notna(match_data['Target_Runs']) and pd.notna(match_data['Target_Overs']) and match_data['Method'] == 'D/L':
        details.append(f'<span class="target-info">Revised Target: {int(match_data["Target_Runs"])} runs in {match_data["Target_Overs"]} overs</span>')
    
    html_content += '<br>'.join(details)
    
    html_content += """
            </div>
        </div>
    </div>
    """
    
    return HTML(html_content)

In [6]:
summary_html = generate_ipl_match_summary(548322)
display(summary_html)

In [8]:
matches[matches['Match_Type'] == 'Final']

,Id,Season,City,Date,Match_Type,Player_Of_Match,Venue,Team1,Team2,Toss_Winner,...,Target_Runs,Target_Overs,Super_Over,Method,Umpire1,Umpire2,match_key,Time,Match_No,_merge
57,336040,2008,Mumbai,2008-06-01,Final,YK Pathan,Dr DY Patil Sports Academy,Chennai Super Kings,Rajasthan Royals,Rajasthan Royals,...,164,20,N,Normal,BF Bowden,RE Koertzen,"(Chennai Super Kings, Rajasthan Royals)",8:00 pm,Match 59,both
114,392239,2009,Johannesburg,2009-05-24,Final,A Kumble,New Wanderers Stadium,Royal Challengers Bangalore,Sunrisers Hyderabad,Royal Challengers Bangalore,...,144,20,N,Normal,RE Koertzen,SJA Taufel,"(Royal Challengers Bangalore, Sunrisers Hydera...",4:30 pm,Match 59,both
174,419165,2010,Mumbai,2010-04-25,Final,SK Raina,Dr DY Patil Sports Academy,Chennai Super Kings,Mumbai Indians,Chennai Super Kings,...,169,20,N,Normal,RE Koertzen,SJA Taufel,"(Chennai Super Kings, Mumbai Indians)",8:00 pm,Match 60,both
247,501271,2011,Chennai,2011-05-28,Final,M Vijay,MA Chidambaram Stadium,Chennai Super Kings,Royal Challengers Bangalore,Chennai Super Kings,...,206,20,N,Normal,Asad Rauf,SJA Taufel,"(Chennai Super Kings, Royal Challengers Bangal...",8:00 pm,Match 74,both
321,548381,2012,Chennai,2012-05-27,Final,MS Bisla,MA Chidambaram Stadium,Kolkata Knight Riders,Chennai Super Kings,Chennai Super Kings,...,191,20,N,Normal,BF Bowden,SJA Taufel,"(Chennai Super Kings, Kolkata Knight Riders)",8:00 pm,Match 76,both
397,598073,2013,Kolkata,2013-05-26,Final,KA Pollard,Eden Gardens,Chennai Super Kings,Mumbai Indians,Mumbai Indians,...,149,20,N,Normal,HDPK Dharmasena,SJA Taufel,"(Chennai Super Kings, Mumbai Indians)",8:00 pm,Match 76,both
457,734049,2014,Bangalore,2014-06-01,Final,MK Pandey,M Chinnaswamy Stadium,Kolkata Knight Riders,Punjab Kings,Kolkata Knight Riders,...,200,20,N,Normal,HDPK Dharmasena,BNJ Oxenford,"(Kolkata Knight Riders, Punjab Kings)",8:00 pm,Match 60,both
516,829823,2015,Kolkata,2015-05-24,Final,RG Sharma,Eden Gardens,Mumbai Indians,Chennai Super Kings,Chennai Super Kings,...,203,20,N,Normal,HDPK Dharmasena,RK Illingworth,"(Chennai Super Kings, Mumbai Indians)",8:00 pm,Match 60,both
576,981019,2016,Bangalore,2016-05-29,Final,BCJ Cutting,M Chinnaswamy Stadium,Royal Challengers Bangalore,Sunrisers Hyderabad,Sunrisers Hyderabad,...,209,20,N,Normal,HDPK Dharmasena,BNJ Oxenford,"(Royal Challengers Bangalore, Sunrisers Hydera...",8:00 pm,Match 60,both
635,1082650,2017,Hyderabad,2017-05-21,Final,KH Pandya,Rajiv Gandhi International Stadium,Mumbai Indians,Rising Pune Supergiant,Mumbai Indians,...,130,20,N,Normal,NJ Llong,S Ravi,"(Mumbai Indians, Rising Pune Supergiant)",8:00 pm,Match 60,both


In [9]:
summary_html = generate_ipl_match_summary(336040)
display(summary_html)